In [56]:
import pandas as pd
import os
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from itertools import combinations,product
import numpy as np
import glob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import l1_min_c
from sklearn import linear_model
from time import time
from sklearn.metrics import recall_score,average_precision_score,precision_score,classification_report
import  matplotlib.pyplot as plt
%matplotlib inline

In [18]:
def generate_aspect_df(df,ignore_aspects=["_id","product_type","id","image","name"],image_aspects=["image","name"]):
    
    d= dict()
    d2=dict()
    
    for i in df.doc:
        l=i.split( )
        for i in l: 
            try:
                col,val = i.split("-",1)
            except Exception as e:
                #print(e)
                #print(i,i.split("-",1))
                pass

            if col in ignore_aspects:
                continue
            if col not in d.keys() :
                d[col]=set() 
            d[col].add(val)         
    d_row =dict.fromkeys(set(d.keys())-set(ignore_aspects))
    for i in d_row:
        d_row[i]=[]

    d_images={}   
    for aspect in d:
        d_images[aspect]={}
        for val in d[aspect]:
            d_images[aspect][val]=[]

    rows = []
    for i in df.doc:
        s= set()
        row=[]
        cols_vals=[] 
        d_image_aspects= dict()
        for ii in i.split(): 
            try:
                col,val = ii.split("-",1)
            except Exception as e:
                #print(e)
                #print(ii,ii.split("-",1))
                continue              
            
            if col in image_aspects:
                if col == "name":
                    val = val.replace("_"," ")
                d_image_aspects[col]=val

                continue
            if col not in ignore_aspects:                
                row.append(ii)
                cols_vals.append((col,val))

            if col not in s and col not in ignore_aspects:
                d_row[col].append(val)
                s.add(col)                        
        rows.append(' '.join(row))
        
        for c,v in cols_vals:           
            if len(d_images[c][v]) == 3:                               
                d_images[c][v][-1]=d_image_aspects
            else:
                d_images[c][v].append(d_image_aspects)               
           
            
        for jj in set(d.keys())-s:
            d_row[jj].append("Unknown")

    vec = CountVectorizer(token_pattern=u"[0-9a-zA-Z_]+-[0-9a-z_]+")
    vec.set_params(binary=True)
    print(rows[0])
    X = vec.fit_transform(rows)
    df_2 = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())                  
    df_2['class'] = df["class"]
    return df_2 ,d_images 

In [19]:
def generate_aspect_df_dummies(df,ignore_aspects=["_id","id","product_type"]):    
    vec = CountVectorizer(token_pattern=u"[0-9a-zA-Z_]+-[0-9a-z_]+")
    vec.set_params(binary=True)
    X = vec.fit_transform(df.doc)
    df_2 = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())              
    df_2['class'] = df["class"]
    return df_2  

In [20]:
def show_aspects (df_2):
    for k in df_2.keys():
        print("{} : {}".format(k, df_2[k].unique())) 

In [21]:
def worsheet_write_row_images(book,sheet_name,mylist,row,col):
    sheet = book.active
    print (mylist)
    mylist = [i  for i in mylist if i is not None]

    for col_delta, img in enumerate(mylist, start=0):        
        cell = sheet.cell(row=row, column=col+col_delta)
        img.anchor = cell.coordinate
        sheet.add_image(img)

In [22]:
def generate_images_workbook(rows,cols,filename):
    book = Workbook()    
    sheet = book.active
    sheet.row_dimensions[1].height = 100  # In pixels

    start_row = 2
    start_col = 2
    start_col2 = start_col + 5
    start_col3 = start_col2 + 3

    for i,r in enumerate(rows):
        print (i,r)
        dress_code,gender,product_type,aspect_name,aspect_val,image1_url,image2_url,image3_url,image_name1,image_name2,image_name3=r
        first_list = [dress_code,gender,product_type,aspect_name,aspect_val]
        second_list = [image1_url,image2_url,image3_url]
        third_list = [image_name1,image_name2,image_name3]
        worsheet_write_row_data(book,sheet_name,first_list, start_row + i,start_col)
        worsheet_write_row_data(book,sheet_name,second_list,start_row + i,start_col2)
        worsheet_write_row_data(book,sheet_name,third_list, start_row + i,start_col3)
        
    workbook_write(book,filename)
             

In [23]:
def generate_net_dummies(df_2,thresh=0.95,alpha_power=2):
    
    all_important_features=[] 
    all_reduced_features = []
    reduced_features=[]
    
    d  = dict()
    
    sorted_aspects = list(set(df_2.columns)-set(['class']))
    sorted_aspects.sort()
        
    for k in sorted_aspects: 
        
        aspect_type,aspect_value = k.split("-",1)
        if aspect_type not in d:
            d[aspect_type]={}
            d[aspect_type]["neg"]=[]
            d[aspect_type]["pos"]=[]
            d[aspect_type]["neu"]=[]           

        template=gum.BayesNet()
        g = gum.RangeVariable(k, k,0,1)
        template.add(g)  
        g_t = gum.RangeVariable("class", "class",0,1)
        template.add(g_t)   
        learner = gum.BNLearner("prods_aspects_temp_dummies.csv", template)
        learner.setSliceOrder([[k],["class"]])
        bn = learner.learnBN()
        arcs = bn.arcs()
        
        if len(arcs) != -1:
            important_features,reduced_features =  get_important_features_dummies(bn,df_2,thresh,k,alpha_power)
            _,n_neg,n_pos,n_neu = reduced_features[0]

            d[aspect_type]["neg"].extend(n_neg)
            d[aspect_type]["pos"].extend(n_pos)
            d[aspect_type]["neu"].extend(n_neu)
            all_important_features.extend(important_features)
            all_reduced_features.extend(reduced_features)
        else:
            
            all_important_features.extend([(aspect_type,aspect_value,0,"no effect")])
            d[aspect_type]["neg"].extend([])
            d[aspect_type]["pos"].extend([])
            d[aspect_type]["neu"].extend([aspect_value])
            all_reduced_features.extend([(aspect_type,[],[],[])])
            
    for aspect in d:
        reduced_features.append((aspect,d[aspect]["neg"],d[aspect]["pos"],d[aspect]["neu"]))
            
    df,df_r = pd.DataFrame(all_important_features,columns=["aspect","value","k","pos_neg","n","n_dress_code","n_feature","n_dress_code_feature"]),pd.DataFrame(reduced_features,columns=["aspect","neg","pos","neu"])                         
    return bn,df,df_r

In [24]:
def lasso(df_2,y_feature):
    feature_names = list(df_2.columns)
    feature_names.remove("class")

    X=df_2[feature_names]
    y=df_2[y_feature]
    start = time()

    cs = l1_min_c(X, y, loss='log') * np.logspace(3, 7, 16)
    print("This took %0.3fs" % (time() - start))

    print("cs",cs)
    print("Computing regularization path ...")
    clf = linear_model.LogisticRegression(penalty='l1', solver='saga',
                                          tol=1e-1, max_iter=int(1e6),
                                         warm_start=False,n_jobs=-1)
    coefs_ = []
    lasso_features = []
    for c in cs:
        clf.set_params(C=c)
        clf.fit(X, y)
        y_pred = clf.predict(X)
        recall,precision = recall_score(y,y_pred),precision_score(y,y_pred)   
        coefs_.append(clf.coef_.ravel().copy())
        mylist = list(zip(feature_names,clf.coef_.ravel().copy()))
        mylist.sort(key= lambda x : abs(x[1]),reverse=True)
        mylist = [x  for x in mylist if abs(x[1])>0.01 ]
        mylist= [('precision',precision),('recall',recall)]+mylist
        lasso_features.append(mylist)
        
        print("precision,recall features",c,mylist)
    return lasso_features
        


In [26]:
def plot_lasso_roc(lasso_features):
    y=[]
    x=[]
    print(lasso_features[0][0][1])
    print('-----------')
    for i in lasso_features:
        
        y.append(i[0][1])
        x.append(i[1][1])
    print(x)
    print(y)
    plt.scatter(x,y) 
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    

In [27]:
def get_highest_score_lasso(lasso_features,precision_w  = 0.9,recall_w =0.1):
    highest_score=-np.inf
    lasso_selected_featured=None
    for i in lasso_features:
        score = precision_w * i[0][1] + recall_w * i[1][1]
        if score > highest_score:
            lasso_selected_featured=i
            highest_score = score 
    return lasso_selected_featured,highest_score
    

In [28]:
def get_important_features_dummies(bn,df_2,thresh = 1,feature="activity",alpha_power=2):    
    important_features = []
    reduced_features=[]
    neg=[]
    pos=[]
    neu=[]   
    
    aspect_type,aspect_value = feature.split("-",1)    
    p = gum.getPosterior(bn, {}, "class").toarray()
    
    # lower as probability of prior higher
    thresh_curr = thresh
    alpha = 10 ** (-alpha_power*p[1])
    thresh = thresh * alpha
    print("aspect:{},thresh_curr:{},alpha:{},thresh:{}".format(feature,thresh_curr,alpha,thresh))
    n = df_2.shape[0]
    n_1 = df_2[df_2["class"]==1].shape[0]
    n_k = df_2[(df_2[feature]==1)].shape[0]
    n_1_k = df_2[(df_2["class"]==1)&(df_2[feature]==1)].shape[0]
    print("feature:{} total:{} dress_code:{} total_with_feature:{} dress_code_with_feature:{}".format(feature,n,n_1,n_k,n_1_k))

    
    for s in [feature]:
        if s == "Unknown":
            continue
        if bn.arcs()==0:
            print("No Effect")
        post_s = gum.getPosterior(bn, {feature:[0,1]}, "class").toarray()
        post_s2= [max(n_1/n,10**-10),max(n_1_k/n_k,10**-10)]
        k = (np.log(post_s)-np.log(p)) 

        print('prior:{},posterior:{},aspect:{},value:{},k:{}'.format(p,post_s,aspect_type,aspect_value,k))     
        post_s[1]=post_s2[1]
        k = (np.log(post_s)-np.log(p)) 
        print("aspect :{},prior {}->{},k{}".format(feature,post_s2[0],post_s2[1],k))

        #print('relative',post_s[1],p[1],(post_s[1]-p[1])/(1-p[1]))
        if abs(k[1]) > thresh  and k[1] < 0 :
            pos_neg = "negative"
            pos.append(aspect_value)
            important_features.append((aspect_type,aspect_value,k,pos_neg,n,n_1,n_k,n_1_k))
        elif abs(k[1]) > thresh and k[1] > 0 :
            pos_neg = "positive"
            neg.append(aspect_value)
            important_features.append((aspect_type,aspect_value,k,pos_neg,n,n_1,n_k,n_1_k))
        else:
            pos_neg = "neutral"
            important_features.append((aspect_type,aspect_value,k,pos_neg,n,n_1,n_k,n_1_k))
            neu.append(aspect_value)
    
    reduced_features.append((aspect_type,pos,neg,neu)) 
    return (important_features,reduced_features)

In [90]:
def generate_aspect_effects(thresh=0.6,alpha_power=2):
    dfs=[]
    dfs_r=[]
    d_images_all=[]
    images_rows = []
    images_cols = ["dress code","gender","product type","aspect name","aspect val","image1","image2","image3","name1","name2","name3"]    
    filenames = glob.glob('../data_docs_with_id/*.csv')
    #remove unisex
    filenames = [filename for filename in filenames if "unisex" not in filename]
    for filename in filenames:
        print ('filename:{}'.format(filename))
        head, tail = os.path.split(filename)
        group =      tail.split('#')
        product =    group[0]
        gender =     group[1]
        dress_code = group[2].split('.')[0].replace("dress_code_","")

        df = pd.read_csv(filename)
        print ("dress code size:{}".format(df[df["class"] == 1].shape[0]))
        
        rows=[]
        df_2,d_images = generate_aspect_df(df)
                
        for aspect_name in d_images:
            for aspect_val in d_images[aspect_name]:
                imgs = d_images[aspect_name][aspect_val]
                n = 3 - len(imgs)
                for i in range(n):
                    imgs.append({'image':'','name':''})
                images_rows.append((dress_code,gender,product,aspect_name,aspect_val,imgs[0]["image"],imgs[1]["image"],imgs[2]["image"],imgs[0]["name"],imgs[1]["name"],imgs[2]["name"]))        
        df_2.to_csv("prods_aspects_temp_dummies.csv")
        bn,df_aspects,df_aspects_r = generate_net_dummies(df_2,thresh,alpha_power)
        #lasso_features = lasso(df_2,"class")
        #plot_lasso_roc(lasso_features)
        #high_lasso_features,highest_lasso_score = get_highest_score_lasso(lasso_features)
        
      
         
        df_aspects["gender"]=gender
        df_aspects["product_type"]= product
        df_aspects["dress_code"]=dress_code

        df_aspects_r["gender"]=gender 
        df_aspects_r["product_type"]=product
        df_aspects_r["dress_code"]=dress_code
        
        dfs.append(df_aspects.copy())
        dfs_r.append(df_aspects_r.copy())
        
    df_images = pd.DataFrame(data=images_rows,columns=["dress_code","gender","product_type","aspect","value","image1","image2","image3","name1","name2","name3"])
    df_images.to_csv("bnf_images_"+str(thresh) + ".csv")
   

    df_all = pd.concat(dfs)
    df_all_r = pd.concat(dfs_r)  
   
    df_all_images  = df_all.merge(df_images,how='inner',on=["dress_code","gender","product_type","aspect","value"])
    df_all_images["y/n"]=''
    # google sheet pic view  from url command
    df_all_images['image1']=df_all_images['image1'].apply(lambda x : '=image("{}")'.format(x))
    df_all_images['image2']=df_all_images['image2'].apply(lambda x : '=image("{}")'.format(x))
    df_all_images['image3']=df_all_images['image3'].apply(lambda x : '=image("{}")'.format(x))
    del(df_all_images['k'])
    df_all_images = df_all_images [['product_type','gender','aspect', 'value', 'pos_neg', 'product_type', 'dress_code',
       'image1', 'image2', 'image3', 'name1', 'name2', 'name3', 'y/n']]
    df_all.to_csv("shirts_bnf_dummiesZI_" +str(thresh) + ".csv")
    df_all_r.to_csv("shirts_bnf_r_dummiesZI_"+str(thresh) +".csv")
    df_all_images.to_csv("shirts_bnf_all_imagesZI_" +str(thresh) + ".csv")
    return df_all,df_all_r,df_all_images

In [91]:
df_all,df_all_r,df_all_images = generate_aspect_effects(1,2)

filename:../data_docs_with_id_intersection_free/shirt#female#dress_code_active.csv
dress code size:124
fabric-denim fabric-lace color-blue color_group-bright color_group-cool_color style-sporty style-fun cut-cropped cut-button
aspect:activity-gym,thresh_curr:1,alpha:0.05048065716667471,thresh:0.05048065716667471
feature:activity-gym total:191 dress_code:124 total_with_feature:22 dress_code_with_feature:22
prior:[0.3515625 0.6484375],posterior:[0.01111111 0.98888889],aspect:activity,value:gym,k:[-3.4544419   0.42201636]
aspect :activity-gym,prior 0.6492146596858639->1.0,k[-3.4544419   0.43318966]
aspect:activity-outdoor,thresh_curr:1,alpha:0.05048065716667471,thresh:0.05048065716667471
feature:activity-outdoor total:191 dress_code:124 total_with_feature:7 dress_code_with_feature:7
prior:[0.3515625 0.6484375],posterior:[0.03333333 0.96666667],aspect:activity,value:outdoor,k:[-2.35582961  0.3992881 ]
aspect :activity-outdoor,prior 0.6492146596858639->1.0,k[-2.35582961  0.43318966]
aspect:

feature:fabric-lycra total:191 dress_code:124 total_with_feature:41 dress_code_with_feature:41
prior:[0.3515625 0.6484375],posterior:[0.0060241 0.9939759],aspect:fabric,value:lycra,k:[-4.06662001  0.42714734]
aspect :fabric-lycra,prior 0.6492146596858639->1.0,k[-4.06662001  0.43318966]
aspect:fabric-mesh,thresh_curr:1,alpha:0.05048065716667471,thresh:0.05048065716667471
feature:fabric-mesh total:191 dress_code:124 total_with_feature:7 dress_code_with_feature:7
prior:[0.3515625 0.6484375],posterior:[0.03333333 0.96666667],aspect:fabric,value:mesh,k:[-2.35582961  0.3992881 ]
aspect :fabric-mesh,prior 0.6492146596858639->1.0,k[-2.35582961  0.43318966]
aspect:fabric-nylon,thresh_curr:1,alpha:0.05048065716667471,thresh:0.05048065716667471
feature:fabric-nylon total:191 dress_code:124 total_with_feature:18 dress_code_with_feature:17
prior:[0.3515625 0.6484375],posterior:[0.06756757 0.93243243],aspect:fabric,value:nylon,k:[-1.64925941  0.36323107]
aspect :fabric-nylon,prior 0.6492146596858639

feature:style-elegant total:191 dress_code:124 total_with_feature:2 dress_code_with_feature:0
prior:[0.3515625 0.6484375],posterior:[0.9 0.1],aspect:style,value:elegant,k:[ 0.94000726 -1.86939544]
aspect :style-elegant,prior 0.6492146596858639->1e-10,k[  0.94000726 -22.59266127]
aspect:style-feminine,thresh_curr:1,alpha:0.05048065716667471,thresh:0.05048065716667471
feature:style-feminine total:191 dress_code:124 total_with_feature:1 dress_code_with_feature:0
prior:[0.3515625 0.6484375],posterior:[0.3515625 0.6484375],aspect:style,value:feminine,k:[0. 0.]
aspect :style-feminine,prior 0.6492146596858639->1e-10,k[  0.         -22.59266127]
aspect:style-fun,thresh_curr:1,alpha:0.05048065716667471,thresh:0.05048065716667471
feature:style-fun total:191 dress_code:124 total_with_feature:1 dress_code_with_feature:1
prior:[0.3515625 0.6484375],posterior:[0.3515625 0.6484375],aspect:style,value:fun,k:[0. 0.]
aspect :style-fun,prior 0.6492146596858639->1.0,k[0.         0.43318966]
aspect:style-o

aspect:cut-snug,thresh_curr:1,alpha:0.19179102616724888,thresh:0.19179102616724888
feature:cut-snug total:98 dress_code:35 total_with_feature:1 dress_code_with_feature:0
prior:[0.64141414 0.35858586],posterior:[0.64141414 0.35858586],aspect:cut,value:snug,k:[0. 0.]
aspect :cut-snug,prior 0.35714285714285715->1e-10,k[  0.         -22.00026378]
aspect:cut-tieback,thresh_curr:1,alpha:0.19179102616724888,thresh:0.19179102616724888
feature:cut-tieback total:98 dress_code:35 total_with_feature:1 dress_code_with_feature:1
prior:[0.64141414 0.35858586],posterior:[0.64141414 0.35858586],aspect:cut,value:tieback,k:[0. 0.]
aspect :cut-tieback,prior 0.35714285714285715->1.0,k[0.         1.02558715]
aspect:cut-tieside,thresh_curr:1,alpha:0.19179102616724888,thresh:0.19179102616724888
feature:cut-tieside total:98 dress_code:35 total_with_feature:2 dress_code_with_feature:0
prior:[0.64141414 0.35858586],posterior:[0.64141414 0.35858586],aspect:cut,value:tieside,k:[0. 0.]
aspect :cut-tieside,prior 0.3

feature:sleeve_style-long_sleeve total:98 dress_code:35 total_with_feature:14 dress_code_with_feature:0
prior:[0.64141414 0.35858586],posterior:[0.98275862 0.01724138],aspect:sleeve_style,value:long_sleeve,k:[ 0.4266882  -3.03485586]
aspect :sleeve_style-long_sleeve,prior 0.35714285714285715->1e-10,k[  0.4266882  -22.00026378]
aspect:sleeve_style-short_sleeve,thresh_curr:1,alpha:0.19179102616724888,thresh:0.19179102616724888
feature:sleeve_style-short_sleeve total:98 dress_code:35 total_with_feature:9 dress_code_with_feature:8
prior:[0.64141414 0.35858586],posterior:[0.13157895 0.86842105],aspect:sleeve_style,value:short_sleeve,k:[-1.5840683   0.88450856]
aspect :sleeve_style-short_sleeve,prior 0.35714285714285715->0.8888888888888888,k[-1.5840683   0.90780412]
aspect:sleeve_style-sleeveless,thresh_curr:1,alpha:0.19179102616724888,thresh:0.19179102616724888
feature:sleeve_style-sleeveless total:98 dress_code:35 total_with_feature:2 dress_code_with_feature:0
prior:[0.64141414 0.35858586]

...

aspect:fabric-cashmere,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fabric-cashmere total:15561 dress_code:35 total_with_feature:2 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:fabric,value:cashmere,k:[0. 0.]
aspect :fabric-cashmere,prior 0.002249212775528565->1e-10,k[  0.        -16.9427963]
aspect:fabric-chiffon,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fabric-chiffon total:15561 dress_code:35 total_with_feature:2 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:fabric,value:chiffon,k:[0. 0.]
aspect :fabric-chiffon,prior 0.002249212775528565->1e-10,k[  0.        -16.9427963]
aspect:fabric-corduroy,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fabric-corduroy total:15561 dress_code:35 total_with_feature:52 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:fabric,value:

aspect:fabric-polyesterlining,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fabric-polyesterlining total:15561 dress_code:35 total_with_feature:79 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:fabric,value:polyesterlining,k:[0. 0.]
aspect :fabric-polyesterlining,prior 0.002249212775528565->1e-10,k[  0.        -16.9427963]
aspect:fabric-poplin,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fabric-poplin total:15561 dress_code:35 total_with_feature:406 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:fabric,value:poplin,k:[0. 0.]
aspect :fabric-poplin,prior 0.002249212775528565->1e-10,k[  0.        -16.9427963]
aspect:fabric-rayon,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fabric-rayon total:15561 dress_code:35 total_with_feature:160 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812

aspect:fit-long_line,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fit-long_line total:15561 dress_code:35 total_with_feature:182 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:fit,value:long_line,k:[0. 0.]
aspect :fit-long_line,prior 0.002249212775528565->1e-10,k[  0.        -16.9427963]
aspect:fit-loose,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fit-loose total:15561 dress_code:35 total_with_feature:761 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[9.99671701e-01 3.28299409e-04],aspect:fit,value:loose,k:[ 0.00195545 -1.93852991]
aspect :fit-loose,prior 0.002249212775528565->1e-10,k[ 1.95545037e-03 -1.69427963e+01]
aspect:fit-low_rise,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:fit-low_rise total:15561 dress_code:35 total_with_feature:2 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:fit,valu

aspect:neckline-collar,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:neckline-collar total:15561 dress_code:35 total_with_feature:5143 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[9.99951395e-01 4.86050355e-05],aspect:neckline,value:collar,k:[ 2.23519747e-03 -3.84872879e+00]
aspect :neckline-collar,prior 0.002249212775528565->1e-10,k[ 2.23519747e-03 -1.69427963e+01]
aspect:neckline-collarless,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:neckline-collarless total:15561 dress_code:35 total_with_feature:18 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:neckline,value:collarless,k:[0. 0.]
aspect :neckline-collarless,prior 0.002249212775528565->1e-10,k[  0.        -16.9427963]
aspect:neckline-crew_neck,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:neckline-crew_neck total:15561 dress_code:35 total_with_feature:9 dress_code_with_feature:0
prior:[

aspect:sleeve_style-kimonosleeve,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:sleeve_style-kimonosleeve total:15561 dress_code:35 total_with_feature:2 dress_code_with_feature:0
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:sleeve_style,value:kimonosleeve,k:[0. 0.]
aspect :sleeve_style-kimonosleeve,prior 0.002249212775528565->1e-10,k[  0.        -16.9427963]
aspect:sleeve_style-long_sleeve,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:sleeve_style-long_sleeve total:15561 dress_code:35 total_with_feature:2897 dress_code_with_feature:4
prior:[0.9977188 0.0022812],posterior:[0.9977188 0.0022812],aspect:sleeve_style,value:long_sleeve,k:[0. 0.]
aspect :sleeve_style-long_sleeve,prior 0.002249212775528565->0.001380738695201933,k[ 0.         -0.50208201]
aspect:sleeve_style-mid_sleeve,thresh_curr:1,alpha:0.9895496839344744,thresh:0.9895496839344744
feature:sleeve_style-mid_sleeve total:15561 dress_code:35 total_with_fea

In [ ]:
df_all_intersection = df_all.copy()


In [96]:
df_all_images_labeled_business_formal = pd.read_csv("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - business formal (1).csv")
df_all_images_labeled_business_formal = df_all_images_labeled_business_formal[["product_type","gender","aspect","value","pos_neg","dress_code","Yes/No (correct/wrong)","Positive/Negative/Ignore "]]
df_all_images_labeled_business_formal = df_all_images_labeled_business_formal.dropna()
df_all_images_labeled_business_formal = df_all_images_labeled_business_formal.rename(index=str, columns={"Positive/Negative/Ignore ": "pos_neg_label", "Yes/No (correct/wrong)": "yes_no"})
df_all_images_labeled_business_formal["pos_neg_label"]= df_all_images_labeled_business_formal["pos_neg_label"].apply(lambda x : x.lower().strip())
df_all_images_labeled_business_formal["yes_no"]=df_all_images_labeled_business_formal["yes_no"].apply(lambda x : x.lower())
df_new_business_formal = df_all_images_labeled_business_formal.merge(df_all,how="inner",on=["dress_code","aspect","value","gender"])
df_new_business_formal["label"] = df_new_business_formal["pos_neg_label"]
df_new_business_formal["label"] = df_new_business_formal["label"].apply(lambda x : "neutral" if x == "ignore" else x)
df_new_business_formal["pred"] = df_new_business_formal["pos_neg_y"]
table = pd.crosstab (df_new_business_formal["label"],df_new_business_formal["pred"])
f=classification_report(df_new_business_formal["label"],df_new_business_formal["pred"])
print(table)
print(f)

pred      negative  neutral  positive
label                                
negative       118        3         6
neutral        122       31        16
positive        47       14        14
              precision    recall  f1-score   support

    negative       0.41      0.93      0.57       127
     neutral       0.65      0.18      0.29       169
    positive       0.39      0.19      0.25        75

   micro avg       0.44      0.44      0.44       371
   macro avg       0.48      0.43      0.37       371
weighted avg       0.51      0.44      0.38       371



In [93]:
def stats_dress_code(filename):
    df_all_images_labeled_dress_code = pd.read_csv(filename)
    df_all_images_labeled_dress_code = df_all_images_labeled_dress_code[["product_type","gender","aspect","value","pos_neg","dress_code","Yes/No (correct/wrong)","Positive/Negative/Ignore "]]
    df_all_images_labeled_dress_code = df_all_images_labeled_dress_code.dropna()
    df_all_images_labeled_dress_code = df_all_images_labeled_dress_code.rename(index=str, columns={"Positive/Negative/Ignore ": "pos_neg_label", "Yes/No (correct/wrong)": "yes_no"})
    df_all_images_labeled_dress_code["pos_neg_label"]= df_all_images_labeled_dress_code["pos_neg_label"].apply(lambda x : x.lower().strip())
    df_all_images_labeled_dress_code["yes_no"]=df_all_images_labeled_dress_code["yes_no"].apply(lambda x : x.lower())
    df_merged_dress_code = df_all_images_labeled_dress_code.merge(df_all,how="inner",on=["dress_code","aspect","value","gender"])
    df_merged_dress_code["label"] = df_merged_dress_code["pos_neg_label"]
    df_merged_dress_code["label"] = df_merged_dress_code["label"].apply(lambda x : "neutral" if x == "ignore" else x)
    df_merged_dress_code["pred"] = df_merged_dress_code["pos_neg_y"]
    table = pd.crosstab (df_merged_dress_code["label"],df_merged_dress_code["pred"])
    f=classification_report(df_merged_dress_code["label"],df_merged_dress_code["pred"])
    #print(table)
    print(f)

In [84]:
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - business formal (1).csv")

              precision    recall  f1-score   support

    negative       0.41      0.93      0.57       138
     neutral       0.67      0.19      0.30       185
    positive       0.41      0.18      0.25        78

   micro avg       0.44      0.44      0.44       401
   macro avg       0.50      0.43      0.37       401
weighted avg       0.53      0.44      0.38       401



In [100]:
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - active .csv")

              precision    recall  f1-score   support

    negative       0.41      0.75      0.53        56
     neutral       0.27      0.05      0.08        60
    positive       0.55      0.58      0.56        59

   micro avg       0.45      0.45      0.45       175
   macro avg       0.41      0.46      0.39       175
weighted avg       0.41      0.45      0.39       175



In [86]:
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - dressy casual.csv")

              precision    recall  f1-score   support

    negative       0.01      0.60      0.03         5
     neutral       0.83      0.34      0.48       275
    positive       0.51      0.40      0.45       152

   micro avg       0.36      0.36      0.36       432
   macro avg       0.45      0.45      0.32       432
weighted avg       0.71      0.36      0.46       432



In [87]:
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - black tie.csv")

              precision    recall  f1-score   support

    negative       0.80      0.95      0.87        77
     neutral       0.00      0.00      0.00         4
    positive       0.83      0.26      0.40        19

   micro avg       0.78      0.78      0.78       100
   macro avg       0.55      0.40      0.42       100
weighted avg       0.78      0.78      0.75       100



In [88]:
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - creative black tie .csv")

              precision    recall  f1-score   support

    negative       0.25      0.99      0.40        94
     neutral       0.74      0.07      0.13       244
    positive       0.45      0.16      0.23        89

   micro avg       0.29      0.29      0.29       427
   macro avg       0.48      0.41      0.25       427
weighted avg       0.57      0.29      0.21       427



In [89]:
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - cocktail.csv")

              precision    recall  f1-score   support

    negative       0.28      0.91      0.43       117
     neutral       0.77      0.10      0.18       235
    positive       0.31      0.10      0.15        81

   micro avg       0.32      0.32      0.32       433
   macro avg       0.46      0.37      0.25       433
weighted avg       0.55      0.32      0.24       433



In [99]:
print("business formal")
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - business formal (1).csv")
print("active")
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - active .csv")
print("coctail")
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - cocktail.csv")
print("creative black tie")
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - creative black tie .csv")
print("dressy casual")
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - dressy casual.csv")
print("black tie")
stats_dress_code("/Users/danielbarkan/Downloads/shirts_bnf_all_imagesZ_1 - black tie.csv")

business formal
              precision    recall  f1-score   support

    negative       0.41      0.93      0.57       127
     neutral       0.65      0.18      0.29       169
    positive       0.39      0.19      0.25        75

   micro avg       0.44      0.44      0.44       371
   macro avg       0.48      0.43      0.37       371
weighted avg       0.51      0.44      0.38       371

active
              precision    recall  f1-score   support

    negative       0.41      0.75      0.53        56
     neutral       0.27      0.05      0.08        60
    positive       0.55      0.58      0.56        59

   micro avg       0.45      0.45      0.45       175
   macro avg       0.41      0.46      0.39       175
weighted avg       0.41      0.45      0.39       175

coctail
              precision    recall  f1-score   support

    negative       0.29      0.91      0.44       111
     neutral       0.77      0.11      0.19       218
    positive       0.31      0.11      0.16 

In [475]:
print(df_new.query("pos_neg_y=='negative' and pos_neg_label=='negative' and n_dress_code_feature==0").shape)
print(df_new.query("pos_neg_y=='negative' and pos_neg_label=='ignore' and n_dress_code_feature==0").shape)
print(df_new.query("pos_neg_y=='negative' and pos_neg_label=='positive' and n_dress_code_feature==0").shape)
print(df_new.query("pos_neg_y=='neutral'  and pos_neg_label=='positive' and n_dress_code_feature==0").shape)
print(df_new.query("pos_neg_y=='neutral'  and pos_neg_label=='negative' and n_dress_code_feature==0").shape)
print(df_new.query("pos_neg_y=='positive' and pos_neg_label=='negative' and n_dress_code_feature==0").shape)
print(df_new.query("pos_neg_y=='positive' and pos_neg_label=='ignore' and n_dress_code_feature==0").shape)

(128, 15)
(133, 15)
(43, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)


In [101]:
"""
df_2 = pd.read_csv("prods_aspects_temp_dummies.csv")

def generate_aspect_effects_(thresh=0.6,alpha_power=2):
    dfs=[]
    dfs_r=[]
    d_images_all=[]
    images_rows = []
    images_cols = ["dress code","gender","product type","aspect name","aspect val","image1","image2","image3","name1","name2","name3"]    
    filenames = glob.glob('../data_docs_with_id/*.csv')
    #remove unisex
    filenames = [filename for filename in filenames if "unisex" not in filename]
    for filename in filenames:
        print ('filename:{}'.format(filename))
        head, tail = os.path.split(filename)
        group =      tail.split('#')
        product =    group[0]
        gender =     group[1]
        dress_code = group[2].split('.')[0].replace("dress_code_","")

        df = pd.read_csv(filename)
        df.head(5)
        print ("dress code size:{}".format(df[df["class"] == 1].shape[0]))
        
        rows=[]
        df_2,d_images = generate_aspect_df(df)
        
        return df,df_2      
        #bn,df_aspects,df_aspects_r = generate_net_dummies(df_2,thresh,alpha_power)
       
d,dd = generate_aspect_effects_()


import gensim
from PIL import Image
import requests

def display_image(url):

    response = requests.get(url, stream=True)
    img = Image.open(response.raw)
    plt.imshow(img)
    plt.show()

def preprocess(text):
    result = []
    tokens = text.split(' ')
    #token_pattern=u"[0-9a-zA-Z_]+-[0-9a-z_]+"
    result = [t for t in tokens if not t.startswith(("id-","image-","name-","product_type-")) ]
    return result
def get_ids_images():
    ids = []
    images=[]
    for text in d['doc']:
        tokens = text.split(' ')
        id   = [t for t in tokens if  t.startswith(("id-")) ]
        image = [t[6:] for t in tokens if  t.startswith(("image-")) ]
        ids = ids + id
        images  = images +image 
    return ids,images
ids,images = get_ids_images()

print(d.head())
print(d.shape)
print(d[d["class"]==1].shape)
d=d[d["class"]==1]
processed_docs= d['doc'].map(preprocess)
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=8
                                       , id2word=dictionary, passes=10, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
print("----------------------------------------------")
for row,bow_vector in enumerate(bow_corpus):
    print ('***',row,ids[row],images[row])
    print(processed_docs[row])
    print
    try :
        display_image(images[row])
    except:
        continue
        
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        print("Topic: {}, Score: {}\t Topic: {}".format(index,score, lda_model.print_topic(index, 5)))
        
        break
 """

'\ndf_2 = pd.read_csv("prods_aspects_temp_dummies.csv")\n\ndef generate_aspect_effects_(thresh=0.6,alpha_power=2):\n    dfs=[]\n    dfs_r=[]\n    d_images_all=[]\n    images_rows = []\n    images_cols = ["dress code","gender","product type","aspect name","aspect val","image1","image2","image3","name1","name2","name3"]    \n    filenames = glob.glob(\'../data_docs_with_id/*.csv\')\n    #remove unisex\n    filenames = [filename for filename in filenames if "unisex" not in filename]\n    for filename in filenames:\n        print (\'filename:{}\'.format(filename))\n        head, tail = os.path.split(filename)\n        group =      tail.split(\'#\')\n        product =    group[0]\n        gender =     group[1]\n        dress_code = group[2].split(\'.\')[0].replace("dress_code_","")\n\n        df = pd.read_csv(filename)\n        df.head(5)\n        print ("dress code size:{}".format(df[df["class"] == 1].shape[0]))\n        \n        rows=[]\n        df_2,d_images = generate_aspect_df(df)\n  